In [12]:
import os
base_dir = "./dataset"
train_dir = os.path.join(base_dir,'train')
test_dir = os.path.join(base_dir,'test')
#print(f'Total number of images in training set:{len(os.listdir(f'{train_dir}'))} ')
print(f'Total number of images in training set:')
print(len(os.listdir(f'{train_dir}')))
print(f'Total number of images in test set: ')
print(len(os.listdir(f'{test_dir}')))

Total number of images in training set:
8227
Total number of images in test set: 
2056


Defining Callback 

In [49]:
from tensorflow.keras.callbacks import Callback
class myCallBack(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') is not None and logs.get('accuracy') > 0.985):
            print(f'\n Reached 98.5% accuracy so training stopped')
            self.model.stop_training = True
"""
class myCallbacks(tf.keras.callbacks.Callback):
    # Define the correct function signature for on_epoch_end
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') is not None and logs.get('accuracy') > 0.985):                 
            print("\nReached 98.5% accuracy so cancelling training!")
                
                # Stop training once the above condition is met
            self.model.stop_training = True"""

'\nclass myCallbacks(tf.keras.callbacks.Callback):\n    # Define the correct function signature for on_epoch_end\n    def on_epoch_end(self, epoch, logs={}):\n        if (logs.get(\'accuracy\') is not None and logs.get(\'accuracy\') > 0.985):                 \n            print("\nReached 98.5% accuracy so cancelling training!")\n                \n                # Stop training once the above condition is met\n            self.model.stop_training = True'

In [41]:
import tensorflow as tf
from tensorflow import keras


# Creating a CNN model for classification of data
model = keras.models.Sequential([
    keras.layers.Conv2D(32,(3,3), activation = 'relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,(3,3), activation = 'relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128,(3,3), activation = 'relu'),
    keras.layers.MaxPooling2D(2,2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(units = 512, activation = 'relu'),
    keras.layers.Dense(units = 7, activation  = 'softmax') #There are 7 different types of sport in the dataset
])

In [42]:
import pandas as pd

#storing the directory of csv files in respective variables
train_csv = './dataset/train.csv'
test_csv = './dataset/test.csv'

#loading the dataframes 
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [43]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   validation_split = 0.25)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_data = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_dir,
    x_col = 'image_ID',
    y_col = 'label',
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True,
    subset = 'training',
    color_mode = 'rgb',
    target_size = (150,150)
)
"""
train_data = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory= train_dir,
    x_col='image_ID',
    y_col='label',
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    shuffle=True,
    subset='training'  # Use training subset
)"""

valid_data = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_dir,
    x_col = 'image_ID',
    y_col = 'label',
    class_mode = 'categorical',
    color_mode = 'rgb',
    target_size = (150,150),
    subset = 'validation',
    shuffle = True,
    batch_size = 32
)

test_data = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = test_dir,
    shuffle = False,
    batch_size = 32,
    class_mode = None,
    x_col = 'image_ID',
    target_size = (150,150)
)

model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
)

Found 6171 validated image filenames belonging to 7 classes.
Found 2056 validated image filenames belonging to 7 classes.
Found 2056 validated image filenames.


In [95]:
#initiating the callbacks class
from tensorflow.keras.callbacks import Callback
#callbacks = myCallback()
history = model.fit(
    train_data,
    epochs = 10,
    validation_data = valid_data,
    #callbacks = [callbacks]
)
sports = {0: 'Badminton', 1: 'Cricket', 2:'Karate' , 3: 'Soccer', 4:'Swimming', 5:'Tennis', 6:'Wrestling'}

Epoch 1/10
193/193 ━━━━━━━━━━━━━━━━━━━━ 60s 310ms/step - accuracy: 0.9852 - loss: 0.0519 - val_accuracy: 0.8142 - val_loss: 1.3379
Epoch 2/10
193/193 ━━━━━━━━━━━━━━━━━━━━ 60s 309ms/step - accuracy: 0.9990 - loss: 0.0045 - val_accuracy: 0.8059 - val_loss: 1.5184
Epoch 3/10
193/193 ━━━━━━━━━━━━━━━━━━━━ 60s 309ms/step - accuracy: 0.9947 - loss: 0.0154 - val_accuracy: 0.8103 - val_loss: 1.6460
Epoch 4/10
 28/193 ━━━━━━━━━━━━━━━━━━━━ 45s 274ms/step - accuracy: 1.0000 - loss: 0.0011

KeyboardInterrupt: 

In [103]:
import numpy as np
import matplotlib as plt
prediction = model.predict(test_data,test_df)
sports = {0:'Badminton', 1:'Cricket', 2: 'Karate', 3:'Soccer', 4:'Swimming',5:'Tennis',6:'Wrestling'}
for i in range(10):
    result = np.argmax(prediction[i])

    print(sports[result])

65/65 ━━━━━━━━━━━━━━━━━━━━ 7s 99ms/step 
Soccer
Tennis
Badminton
Wrestling
Tennis
Tennis
Cricket
Tennis
Swimming
Soccer
